# Extending the Billboard kaggle dataset using API

In [6]:
import pandas as pd

In [50]:
billboard_top100 = pd.read_csv('Hit_charts.csv')
billboard_top100.head()

,rank,song,artist,last-week,peak-rank,weeks-on-board,date
0,1,Butter,BTS,1.0,1,7,17/07/2021
1,2,Good 4 U,Olivia Rodrigo,2.0,1,8,17/07/2021
2,3,Levitating,Dua Lipa Featuring DaBaby,4.0,2,40,17/07/2021
3,4,Kiss Me More,Doja Cat Featuring SZA,3.0,3,13,17/07/2021
4,5,Montero (Call Me By Your Name),Lil Nas X,8.0,1,15,17/07/2021


As we can see in top 5 results, the data was stoped at 17/07/2021 but we have new songs that are in top 100 but are not here. for ex, the song ' Industry Baby ' is in this week's top 100 but it is not in this dataset. 

So i will get the newest data from the Billboard API..

## Billboard API

In [ ]:
#pip install billboard.py

In [5]:
import billboard

In [19]:
chart = billboard.ChartData('hot-100')
chart.date 

'2021-10-23'

In [21]:
song = chart[0]
song.title

'Industry Baby'

now lets loop throw all the billoard's charts, taking only the weeks that were not included in the kaggle dataset..

In [38]:
artist_ = []
title_ = []
peakPos_=[]
date_ = []
dict_ = {}

chart = billboard.ChartData('hot-100')

counter =0 
while chart.previousDate and counter<21:
    counter +=1
    for i in range(100):
        song = chart[i]
        artist_.append(song.artist)
        title_.append(song.title)
        peakPos_.append(song.peakPos)
        date_.append(chart.date)
    chart = billboard.ChartData('hot-100', chart.previousDate)

   
dict_['artist'] = artist_
dict_['song'] = title_
dict_['peakPos_'] = peakPos_ 
dict_['Date'] = date_
charts = pd.DataFrame(dict_)
charts.shape

(2100, 4)

now we got the most recent date's charts.. 

In [37]:
charts

,artist,song,peakPos_,Date
0,Lil Nas X & Jack Harlow,Industry Baby,1,2021-10-23
1,The Kid LAROI & Justin Bieber,Stay,1,2021-10-23
2,Walker Hayes,Fancy Like,3,2021-10-23
3,Ed Sheeran,Bad Habits,2,2021-10-23
4,Drake Featuring Future & Young Thug,Way 2 Sexy,1,2021-10-23
...,...,...,...,...
2095,Thomas Rhett,Country Again,73,2021-06-05
2096,J. Cole,punchin'.the.clock,20,2021-06-05
2097,Morgan Wallen,Wasted On You,9,2021-06-05
2098,J. Cole,the.climb.back,25,2021-06-05


lets add them .. 

In [40]:
billboard_top100 = billboard_top100.drop(['rank','last-week','weeks-on-board'],axis=1)
billboard_top100

,song,artist,peak-rank,date
0,Butter,BTS,1,17/07/2021
1,Good 4 U,Olivia Rodrigo,1,17/07/2021
2,Levitating,Dua Lipa Featuring DaBaby,2,17/07/2021
3,Kiss Me More,Doja Cat Featuring SZA,3,17/07/2021
4,Montero (Call Me By Your Name),Lil Nas X,1,17/07/2021
...,...,...,...,...
328482,Over And Over,Thurston Harris,96,04/08/1958
328483,I Believe In You,Robert & Johnny,97,04/08/1958
328484,Little Serenade,The Ames Brothers,98,04/08/1958
328485,I'll Get By (As Long As I Have You),Billy Williams,99,04/08/1958


In [47]:
billboard_top100.rename(columns = {"peak-rank":"peakPos_","date":"Date"}, inplace = True)

In [48]:
result = charts.append([billboard_top100])
result

,artist,song,peakPos_,Date
0,Lil Nas X & Jack Harlow,Industry Baby,1,2021-10-23
1,The Kid LAROI & Justin Bieber,Stay,1,2021-10-23
2,Walker Hayes,Fancy Like,3,2021-10-23
3,Ed Sheeran,Bad Habits,2,2021-10-23
4,Drake Featuring Future & Young Thug,Way 2 Sexy,1,2021-10-23
...,...,...,...,...
328482,Thurston Harris,Over And Over,96,04/08/1958
328483,Robert & Johnny,I Believe In You,97,04/08/1958
328484,The Ames Brothers,Little Serenade,98,04/08/1958
328485,Billy Williams,I'll Get By (As Long As I Have You),99,04/08/1958


In [51]:
result.to_csv('100_charts.csv')